# Imports

In [1]:
import asyncio
from uuid import uuid4

In [2]:
from pydantic import BaseModel

In [3]:
from matter_persistence.redis.manager import CacheManager, get_connection_pool
from matter_persistence.foundation_model import FoundationModel

# UUIDS

In [4]:
# random organisation and internal id
ORGANISATION_ID = uuid4()
INTERNAL_ID = uuid4()

# Example Pydantic type

In [5]:
# example pydantic type
class Person(FoundationModel):
    name: str
    age: int

# test type
john = Person(name="John", age=30)

In [6]:
john # created_at & created_at_timestamp are added "automatically"

Person(created_at=datetime.datetime(2024, 4, 30, 12, 14, 10, 499668, tzinfo=datetime.timezone.utc), created_at_timestamp=1714479250.499696, name='John', age=30)

# Create Person from different objects

## from python dict

In [7]:
Person.parse_obj({'name': 'Jane', 'age': 30})

Person(created_at=datetime.datetime(2024, 4, 30, 12, 14, 10, 499668, tzinfo=datetime.timezone.utc), created_at_timestamp=1714479250.499696, name='Jane', age=30)

## from other pydantic type

In [8]:
class PersonWithOnlyName(BaseModel):
    name: str

class PersonWithHairColour(BaseModel):
    name: str
    age: int
    hair_colour: str

In [9]:
# the required fields must be present in the other pydantic type!!

In [10]:
Person.parse_obj(PersonWithOnlyName(name="Adam"))

ValidationError: 1 validation error for Person
age
  Field required [type=missing, input_value={'name': 'Adam'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [11]:
Person.parse_obj(PersonWithHairColour(name="Simone", age=30, hair_colour="brown"))  # extra fields are ignored

Person(created_at=datetime.datetime(2024, 4, 30, 12, 14, 10, 499668, tzinfo=datetime.timezone.utc), created_at_timestamp=1714479250.499696, name='Simone', age=30)

# CacheManager

In [12]:
connection_pool = get_connection_pool(host="localhost", port=6379)
cache_manager = CacheManager(connection_pool=connection_pool) # make sure there is a redis instance running!

# Save, get, and delete pydantic type

In [13]:
# check if cache is alive
await cache_manager.is_cache_alive()

True

In [14]:
# save pydantic type to cache
await cache_manager.save_value(ORGANISATION_ID, INTERNAL_ID, john, Person)

In [15]:
# check if pydantic object exists in cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Person)

True

In [16]:
# find pydantic object in cache
await cache_manager.get_value(ORGANISATION_ID, INTERNAL_ID, Person)

CacheRecordModel(created_at=datetime.datetime(2024, 4, 30, 12, 14, 10, 499668, tzinfo=TzInfo(UTC)), created_at_timestamp=1714479250.499696, internal_id='02986cc1-217e-4678-9bf8-7240dee597ed', hash_key='f5cd4bf2-2d1f-4d24-99e4-824765ebccbc_Person_43fc3da42e68a386fa58ff976d2ccdf01796fbf2', organization_id=UUID('f5cd4bf2-2d1f-4d24-99e4-824765ebccbc'), value={'created_at': '2024-04-30T12:14:10.499668Z', 'created_at_timestamp': 1714479250.499696, 'name': 'John', 'age': 30}, expiration=None)

In [17]:
# delete pydantic object from cache
await cache_manager.delete_value(ORGANISATION_ID, INTERNAL_ID, Person)

In [18]:
# make sure pydantic object is deleted from cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Person)

False

# Close connection pool

In [19]:
await cache_manager.close_connection_pool()